# Inflation Data

In [ ]:
df_inflation = pd.read_excel('CPIPctChangeData.xlsx',engine='openpyxl',sheet_name = 'BLS Data Series',skiprows=11)

window = 3
df_inflation['RollingAvg5Yr'] = df_inflation.Annual.rolling(window).mean()
df_inflation['RollingSD5Yr'] = df_inflation.Annual.rolling(window).std()

df_inflation['RollingAvg5Yr_low'] = df_inflation.Annual.rolling(window).min()
df_inflation['RollingAvg5Yr_high'] = df_inflation.Annual.rolling(window).max()

#df_inflation['RollingAvg5Yr_low'] = df_inflation['RollingAvg5Yr'] - df_inflation['RollingSD5Yr']
#df_inflation['RollingAvg5Yr_high'] = df_inflation['RollingAvg5Yr'] + df_inflation['RollingSD5Yr']
df_inflation.head() 

NameError: name 'pd' is not defined

In [ ]:
fig = go.Figure()

y = df_inflation['Annual']
x = df_inflation['Year']
y_roll = df_inflation['RollingAvg5Yr']
y_rollmin = df_inflation['RollingAvg5Yr_low']
y_rollmax = df_inflation['RollingAvg5Yr_high']

fig.add_trace(go.Scatter(x=x, y=y,
                         marker=dict(color='#005100',size = 5),
                         line = dict(color='#005100',width = 1),
                         mode='markers+lines',
                         name='Yearly Inflation Rate'))

fig.add_trace(go.Scatter(x=x, y=y_roll,
                         line = dict(color='#FF3333',width = 1),
                         mode='lines',
                         name='5 Year Rolling Avg'))

fig.add_trace(go.Scatter(x=x, y=y_rollmin,
                         line = dict(color='#bf9fa2',width = 1),
                         mode='lines',
                         name='5 Year Rolling Low'))

fig.add_trace(go.Scatter(x=x, y=y_rollmax,
                         line = dict(color='#bf9fa2',width = 1),
                         mode='lines',
                         fill='tonexty',
                         name='5 Year Rolling High'))



fig.update_xaxes(tickangle=-45, tickfont = dict(family = 'Arial', size = 14,color = 'black'))
fig.update_layout(hovermode="x unified",clickmode ='select',height=500,width=1100,)

fig.show(config=scatter_config)


In [ ]:
#https://machinelearningmastery.com/time-series-forecasting-with-prophet-in-python/
series = df_inflation['Annual']
lag_plot(series)
pyplot.show()

df_model = df_inflation.copy()
df_model = df_model[['Year', 'Annual']]
df_model.rename(columns={"Year": "ds", "Annual": "y"},inplace=True)
df_model['ds']= pd.to_datetime(df_model['ds'],format='%Y')
# define the model
model = Prophet()
# fit the model
model.fit(df_model)

display('Model change points data')
display(model.changepoints_t)

display('Model change points times')
display(model.changepoints)

display('Model Component Modes')
display(model.component_modes)

display('Model Parameters')
display(model.params)

#https://medium.com/analytics-vidhya/time-series-analysis-a-quick-tour-of-fbprophet-cbbfbffdf9d8#:~:text=Prophet%20is%20a%20procedure%20for,several%20seasons%20of%20historical%20data.
future = model.make_future_dataframe(periods= 5, freq= 'Y')
forcast = model.predict(future)
fig = model.plot(forcast)
fig.show()
fig = model.plot_components(forcast)

NameError: name 'df_inflation' is not defined

In [ ]:
#don't run causes memory error attempt at autoarima

'''import pmdarima as pm

model_data = cpi_monthly.copy()
model_data.set_index('Date',inplace = True)
model_data = model_data[['Inflation']].dropna()

display(model_data.head())
# Seasonal - fit stepwise auto-ARIMA
smodel = pm.auto_arima(model_data, start_p=1, start_q=1,
                       test='adf',
                       max_p=3, max_q=3, m=12,
                       start_P=0, seasonal=True,
                       d=None, D=1, trace=True,
                       error_action='ignore',  
                       suppress_warnings=True,
                       stepwise=True)

smodel.summary()'''

# Prophet Modeling

In [ ]:
def build_model_prophet(model_data = None,fourier_order  = None):
    model = Prophet(seasonality_mode='multiplicative',
                        growth= 'linear',
                        weekly_seasonality=False,
                        daily_seasonality=False,
                        yearly_seasonality = False,
                        mcmc_samples=0,
                        changepoint_range=1,
                        changepoint_prior_scale=0.25,
                        #n_changepoints = 2
                        #).add_seasonality(
                        #    name ='monthly',period =30.5,fourier_order = 55
                        #).add_seasonality(
                        #name ='yearly',period =365.25,fourier_order = fourier_order
                        )

    model.fit(model_data)

    future = model.make_future_dataframe(periods= 24, freq= 'M')

    forcast = model.predict(future)
    
    return {'model': model, 'forcast': forcast}

In [ ]:
df_model_cpi = df_cpi.copy()
df_model_cpi = df_model_cpi[['Year', 'CPI']]
display(df_model_cpi.head())
df_model_cpi.rename(columns={"Year": "ds", "Annual": "y"},inplace=True)
df_model_cpi['ds']= pd.to_datetime(df_model_cpi['ds'],format='%Y')

fourier_orders = list(range(0,10,1))

model_dict = {}
display(Markdown('<h2>Model Building in Progress...</h2>'))
counter = 1
'''
for i in fourier_orders:
    if i == 0:
        pass
    else:
        display(Markdown(f'Building {counter}/{len(fourier_orders)-1}'))
        #display(Markdown(f'Order: {i}'))
        temp_model = build_model_prophet(model_data = df_model_cpi,fourier_order = i)
        model = temp_model['model']
        forcast = temp_model['forcast']
        plot_model(model_data = df_model_cpi,model =  model,forcast = forcast,fourier_order = i)
        plot_model_components(model = model,forcast = forcast)
        
        model_dict[f'Order {i}'] = temp_model
        counter += 1

'''
temp_model = build_model_prophet(model_data = df_model_cpi,fourier_order = None)
model = temp_model['model']
forcast = temp_model['forcast']
plot_model(model_data = df_model_cpi,model =  model,forcast = forcast,fourier_order = i)
plot_model_components(model = model,forcast = forcast)

model_dict[f'Order {i}'] = temp_model
        
display(Markdown('<h1>Model Building Completed.</h1>'))

In [ ]:
def build_model_prophet(model_data = None,fourier_order  = None):
    model = Prophet(seasonality_mode='multiplicative',
                        growth= 'linear',
                        weekly_seasonality=False,
                        daily_seasonality=False,
                        yearly_seasonality = False,
                        mcmc_samples=0,
                        changepoint_range=1,
                        changepoint_prior_scale=0.25,
                        #n_changepoints = 5
                        #).add_seasonality(
                        #    name ='monthly',period =30.5,fourier_order = 55
                        ).add_seasonality(
                        name ='yearly',period =365.25,fourier_order = fourier_order
                        )

    model.fit(model_data)

    future = model.make_future_dataframe(periods= 24, freq= 'M')

    forcast = model.predict(future)
    
    return {'model': model, 'forcast': forcast}

In [ ]:
df_model_cpi = df_cpi.copy()
df_model_cpi = df_model_cpi[['Year', 'Inflation']]
df_model_cpi.rename(columns={"Year": "ds", "Inflation": "y"},inplace=True)
df_model_cpi['ds']= pd.to_datetime(df_model_cpi['ds'],format='%Y')

fourier_orders = list(range(0,6,1))

model_dict = {}
display(Markdown('<h2>Model Building in Progress...</h2>'))
counter = 1
for i in fourier_orders:
    if i == 0:
        pass
    else:
        display(Markdown(f'Building {counter}/{len(fourier_orders)-1}'))
        #display(Markdown(f'Order: {i}'))
        temp_model = build_model_prophet(model_data = df_model_cpi,fourier_order = i)
        model = temp_model['model']
        forcast = temp_model['forcast']
        #plot_model(model_data = df_model_cpi,model =  model,forcast = forcast,fourier_order = i)
        #plot_model_components(model = model,forcast = forcast)

        fig = model.plot(forcast, xlabel='Date', ylabel='Inflation Rate (Order: {})'.format(i))
    
        a = add_changepoints_to_plot(fig.gca(), model, forcast)
        fig.show()
        fig = model.plot_components(forcast)
        fig.show()
            
        model_dict[f'Order {i}'] = temp_model
        counter += 1
        
display(Markdown('<h1>Model Building Completed.</h1>'))

## Import item price data

In [ ]:
pricing = pd.read_excel('IndividualPricingData.xls',engine='xlrd',sheet_name = 'Monthly')
pricing['DATE'] = pricing['DATE'].dt.date
rename_map = pd.read_excel('IndividualPricingData.xls',engine='xlrd',sheet_name = 'MatchedMap')

for column in pricing.columns:
    if column == 'DATE':
        #skip column if it's the date column
        continue
    else:
        temp_real_col = rename_map['StripName2'][rename_map['SourceID'] == column].values[0]
        temp_real_col = temp_real_col.split(' (')
        #print(temp_real_col)
        pricing.rename(columns = {column: temp_real_col[0]},inplace = True)

pricing.set_index('DATE',inplace = True)

display(Markdown('<h2>Pricing Data</h2>'))
display(pricing.head())
'''
for column in pricing.columns:
    print(column)
    inflation_name = column + ' INFLATION'
    pricing[inflation_name] = (pricing[column].diff()/ pricing[column].shift(1))*100
'''


In [ ]:
print(len(list(pricing.columns.values)))

print(len(list(set(pricing.columns.values))))

In [ ]:
import collections
dupe_columns = [item for item, count in collections.Counter(list(pricing.columns.values)).items() if count > 1]
display('The following column names are dupliated:')
display(dupe_columns)



In [ ]:
size_prices = {}

for column in pricing.columns:
    temp_series = pricing[column].dropna()
    item_datapoints = len(temp_series)
    size_prices[column] = item_datapoints

df_sizes = pd.DataFrame(size_prices, index=['DataPoints']).T
df_sizes.reset_index(inplace = True)
df_sizes.rename(columns = {'index': 'ItemNames'},inplace = True)

fig = px.bar(df_sizes, x='ItemNames', y='DataPoints')
fig.update_layout(height=700,width=1100,)
display(Markdown('<h2>Number of Data Points per Item</h2>'))
fig.show()


'''item_most_data = max(size_prices, key=size_prices.get)
size_most_data = size_prices[item_most_data]
display(Markdown(f'Item with the most data: **{item_most_data} ({size_most_data} points)**'))

pct_of_max = 0.6
min_num_points = int(pct_of_max*size_most_data)
display(Markdown(f'Removing Variables with < **{min_num_points} points**'))'''

mode_data = stats.mode(df_sizes['DataPoints'])
display(Markdown(f'Mode of data points: **{mode_data} points**'))
display(Markdown(f'Keeping variables with only data points = **{mode_data} points**'))


ideal_data_keys = []
for key in size_prices.keys():
    if size_prices[key] == 506 or size_prices[key] == 505:
        ideal_data_keys.append(key)


pricing_filtered = pricing.copy()
pricing_filtered =  pricing_filtered[ideal_data_keys]
pricing_filtered = pricing_filtered.dropna()
pricing_filtered



In [ ]:
fig = go.Figure()

# Loop df columns and plot columns to the figure
for i in pricing_filtered.columns:
    fig.add_trace(go.Scatter(x=pricing_filtered.index.values, y=pricing_filtered[i],
                        mode='lines', # 'lines' or 'markers'
                        name=i))
display(Markdown('<h2>Price of Goods with the Same Time Period</h2>'))
fig.show()

In [ ]:
pricing_filtered_inf = pricing_filtered.copy()

for column in pricing_filtered_inf.columns:
    inflation_name = column + ' INFLATION'
    pricing_filtered_inf[inflation_name] = (pricing_filtered_inf[column].diff()/ pricing_filtered_inf[column].shift(1))*100

fig = go.Figure()

# Loop df columns and plot columns to the figure
for i in pricing_filtered_inf.columns:
    if 'INFLATION' in i:
        fig.add_trace(go.Scatter(x=pricing_filtered_inf.index.values, y=pricing_filtered_inf[i],
                            mode='lines', # 'lines' or 'markers'
                            name=i))
display(Markdown('<h2>Inflation Rate of Goods with the Same Time Period</h2>'))
fig.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c5ceaca9-e365-4132-a867-b543164c96b0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>